In [3]:
'''
Script: Employee Map Automation
This arcpy tool allows the user to create a map (heatmap) with office sites, numbers of employees by zipcode, 
and a mean center of employee locations. It uses two csv files, which were a office location csv with latitude 
and longitude and a employee csv with zipcode. It creates a mean center point by removing outliers above the 
search radius from a median center point of employee locations. Also, it creates a GDB table and shows employee
counts by NY regions used for a final production work.
Author: Jeong Hoon Kim (JeongHoon.Kim@cbre.com)
Date: May 18, 2022
Last Updated: October 8, 2022
'''
# 0. ENVIRONEMNT
# 0.1 IMPORT MODULES
import arcpy, os, sys
from arcpy import env
# 0.2 ENVIRONMENT OPTIONS
arcpy.env.overwriteOutput = True
# 0.3 PARAMETERS
work_dbs = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb"
off_csv = r"C:\ArcGIS\Projects\Dev_EMA\csv_example\Office_220907_2.csv" # Optional
off_point = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\off_point_test" # Optional
emp_csv = r"C:\ArcGIS\Projects\Dev_EMA\csv_example\Employee_220907.csv"
emp_field = "ZIPCODE"
emp_point = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\emp_point_test"
emp_zipcode = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\emp_zipcode_test"
emp_meancenter = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\emp_meancenter_test"
src_distance = "50 miles"
# 0.4 SET WORKSPACE
arcpy.env.workspace = work_dbs
workspace = work_dbs
# 1. GEOPROCESSING
# 1.1 CREATE OFFICE SITES - Optional
out_coordinate_system = arcpy.SpatialReference('WGS 1984 Web Mercator (auxiliary sphere)')
if off_csv == None:
    print("You did not enter a csv file for office sites!") # arcpy.AddMessage
    pass
else: 
    print("You entered a csv file for office sites!") # arcpy.AddMessage
    try:
        # Set local variable
        off_point_nprj = os.path.join(workspace, "Off_Point_Nprj")
        # Create unprojected feature layer for office sites
        arcpy.management.XYTableToPoint(
            in_table = off_csv,
            out_feature_class = off_point_nprj,
            x_field = "LONG",
            y_field = "LAT",
            coordinate_system = arcpy.SpatialReference("WGS 1984"))
        # Project the layer
        arcpy.Project_management(
            in_dataset = off_point_nprj,
            out_dataset = off_point,
            out_coor_system = out_coordinate_system)
        # Delete unnecessary variable
        arcpy.management.Delete(off_point_nprj)
    except:
        print("ERROR: CREATE LAYER | Office Site") # arcpy.AddError
    else: 
        print("Office site feature is created successfully!") # arcpy.AddMessage
# 1.2 CREATE EMPLOYEE LOCATIONS - Required
isItCSV_emp = emp_csv[-4:]
if isItCSV_emp == '.csv':
    print("You entered a csv file for employee locations!")
    try:
        # Import a pop-centered by zipcode csv file
        popCenter_csv = r"N:\Team-GISData\Researched_Data\NewYork\Arcpy_Tools\EmployeeMap\PopCenterZips_2022_csv.csv"
        # Set local variables
        emp_table = os.path.join(workspace, "emp_table")
        empJoin_table = os.path.join(workspace, "empJoin_table")
        emp_point_nprj = os.path.join(workspace, "Emp_Point_NProj")
        # Copy csv to gdb table
        arcpy.CopyRows_management(emp_csv, emp_table) # to give uniuqe oid to each row
        # Join the two tables with the assigned fields and copy it
        arcpy.JoinField_management(emp_table, emp_field, popCenter_csv, "ZipCode")
        arcpy.Copy_management(emp_table, empJoin_table)
        # Create unprojected feature layer for employee locations
        arcpy.management.XYTableToPoint(
            in_table = empJoin_table, 
            out_feature_class = emp_point_nprj,
            x_field = "Long", 
            y_field = "Lat", 
            coordinate_system = arcpy.SpatialReference("WGS 1984"))
        # Project the feature layer
        arcpy.Project_management(
            in_dataset = emp_point_nprj,
            out_dataset = emp_point,
            out_coor_system = out_coordinate_system)
        # Delete unnecessary objects
        arcpy.management.Delete(emp_table)
        arcpy.management.Delete(empJoin_table)
        arcpy.management.Delete(emp_point_nprj)
    except:
        print("ERROR: CREATE LAYER | Employee Location")
    else:
        print("Employee point feature is created successfully!")
else:
    print("ERROR: INPUT OR FORMAT | Did not entere employee csv or entered wrong format") #arcpy.AddError
# 1.3 MEAN CENTER
if src_distance == None:
    print("You did not enter a search radius for removing outliers of employee locations!")
    try:
        arcpy.MeanCenter_stats(
            Input_Feature_Class = emp_point,
            Output_Feature_Class = emp_meancenter)
    except:
        print("ERROR: CREATE LAYER | Mean Center without Search Radius")
    else:
        print("Mean center point without a search radius is created successfully!")
else:
    print("Your search radius for employee points is "+ src_distance +"!")
    try:
        # Set local variables
        emp_mediancenter = os.path.join(workspace, "emp_mediancenter")
        emp_selected = os.path.join(workspace, "emp_selected")
        # Create a median center point
        arcpy.MedianCenter_stats(
            Input_Feature_Class = emp_point, 
            Output_Feature_Class = emp_mediancenter)
        # Select within the search radius from the median point
        selection = arcpy.SelectLayerByLocation_management(
            in_layer = emp_point,
            overlap_type = "WITHIN_A_DISTANCE_GEODESIC",
            select_features = emp_mediancenter,
            search_distance = src_distance,
            selection_type = "NEW_SELECTION")
        arcpy.CopyFeatures_management(selection, emp_selected)
        # Create a mean center point with the selected employees
        arcpy.MeanCenter_stats(
            Input_Feature_Class = emp_selected, 
            Output_Feature_Class = emp_meancenter)
        # Delete unnecessary objects
        arcpy.management.Delete(emp_mediancenter)
        arcpy.management.Delete(emp_selected)
    except: 
        print("ERROR: CREATE LAYER | Mean Center with Search Radius")
    else: 
        print("Mean center point within the search radius is created successfully!")
# 1.4 ZIPCODE AND COUNT
try: 
    # Set local variables
    zipcodes = r"N:\Team-GISData\Researched_Data\NewYork\Arcpy_Tools\EmployeeMap\EmployeeMap.gdb\ZIPCodes_zp_Gen2_2021"
    nyregion = r"N:\Team-GISData\Researched_Data\NewYork\Arcpy_Tools\EmployeeMap\EmployeeMap.gdb\NYRegions_Count_2022"
    emp_count = os.path.join(workspace, "emp_count_test")
    # Spatial-join with zipcodes - Map
    arcpy.SpatialJoin_analysis(
        target_features = zipcodes, 
        join_features = emp_point, 
        out_feature_class = emp_zipcode, 
        join_operation = "JOIN_ONE_TO_ONE", 
        join_type = 0)
    # Spatial-join with nyregion - Count
    arcpy.SpatialJoin_analysis(
        target_features = nyregion, 
        join_features = emp_point, 
        out_feature_class = emp_count, 
        join_operation = "JOIN_ONE_TO_ONE", 
        join_type = "KEEP_ALL")
    # Copy the nyregion count feature to table with selected fields
    myfields = ['Join_Count', 'ID', 'REGION', 'SUBREGION'] # list of fields to keep
    fms = arcpy.FieldMappings() # create an empty field mappings, create an individual field map for each field, and add it to the field mappings
    for field in myfields :
        fm = arcpy.FieldMap()
        fm.addInputField(emp_count, field)
        fms.addFieldMap(fm)
    arcpy.conversion.TableToTable(
        in_rows = emp_count, 
        out_path = workspace, 
        out_name = "NYRegions_Count_Table", 
        field_mapping = fms)
except:
    print("ERROR: CREATE LAYER | Employees by Zipcode Layer & NY Regional Counts")
else:
    print("Employees by zipcode layer and NY regional counts table are created successfully!")
# 2. MAP
# 2.1 ADD LAYERS TO MAP
# Set current project, map and local variable
aprx = arcpy.mp.ArcGISProject("CURRENT") # Current Project
aprxMap = aprx.listMaps()[0] # Default Map
cbreMap = r"N:\Team-GISData\Distributed_Data\Base Layers\Basemaps\Map Style Layer Files\2022\Neutral Basemap BA Update Pro2.9 2022.lyrx" # CBRE neutral basemap
nyregion_table = os.path.join(workspace, "NYRegions_Count_Table")
# Import layers to map
if off_csv == None:
    try: 
        import_layers = [cbreMap, emp_zipcode, emp_meancenter, nyregion_table]
        for layer in import_layers:
            aprxMap.addDataFromPath(layer)
    except:
        print("ERROR: ADD LAYER TO MAP | without Office")
    else:
        print("Total 4 layers without office are added to map successfully!")
else:
    try:
        import_layers = [cbreMap, emp_zipcode, emp_meancenter, off_point, nyregion_table]
        for layer in import_layers:
            aprxMap.addDataFromPath(layer)
    except:
        print("ERROR: ADD LAYER TO MAP | with Office")
    else:
        print("Total 5 layers with office are added to map successfully!")      
# 2.2 RENDERING
# 1) EMPLOYEE BY ZIPCODE
try: 
    lyr_zipcode = aprxMap.listLayers("*zip*")[0]
    sym = lyr_zipcode.symbology
    if hasattr(sym, 'renderer'):
        if sym.renderer.type == 'SimpleRenderer':
            sym.updateRenderer('GraduatedColorsRenderer')
            sym.renderer.classificationField = 'Join_Count'
            sym.renderer.classificationMethod = 'NaturalBreaks'
            sym.renderer.breakCount = 5
            sym.renderer.colorRamp = aprx.listColorRamps('Yellow-Orange-Red (Continuous)')[0]        
            lyr_zipcode.symbology = sym
    lyr_zipcode.showLabels = True
    lyr_zipcode.transparency = 20
except:
    print("ERROR: RENDERING | Employee by Zipcode")
else:
    print("Rendering for employee by zipcode is completed successfully!")
# 2) MEAN CENTER
try:
    lyr_meanCenter = aprxMap.listLayers("*mean*")[0]
    sym2 = lyr_meanCenter.symbology
    if hasattr(sym2, 'renderer'):
        if sym2.renderer.type == 'SimpleRenderer':
            sym2.renderer.symbol.applySymbolFromGallery("MeanCenter", 0)
            sym2.renderer.symbol.size = 20
            lyr_meanCenter.symbology = sym2
except:
    print("ERROR: RENDERING | Employee Mean Center")
else:
    print("Rendering for employee mean center is completed successfully!")
# 3) OFFICE
try:
    lyr_office = aprxMap.listLayers("*off*")[0]
    sym3 = lyr_office.symbology
    if hasattr(sym3, 'renderer'):
        if sym3.renderer.type == 'SimpleRenderer':
            sym3.updateRenderer('UniqueValueRenderer')
            sym3.renderer.fields = ["NAME"]
            for grp in sym3.renderer.groups:
                for itm in grp.items:
                    officeName = itm.values[0][0]
                    if officeName == "Office 1":
                        itm.symbol.applySymbolFromGallery("Office_1", 0)
                        itm.symbol.size = 20
                    if officeName == "Office 2":
                        itm.symbol.applySymbolFromGallery("Office_2", 0)
                        itm.symbol.size = 20
                    if officeName == "Office 3":
                        itm.symbol.applySymbolFromGallery("Office_3", 0)
                        itm.symbol.size = 20
                    if officeName == "Office 4":
                        itm.symbol.applySymbolFromGallery("Office_4", 0)
                        itm.symbol.size = 20
                    if officeName == "Office 5":
                        itm.symbol.applySymbolFromGallery("Office_5", 0)
                        itm.symbol.size = 20
            lyr_office.symbology = sym3
except:
    print("ERROR: RENDERING | Office Sites")
else:
    print("Rendering for office sites is completed successfully!")
# 2.3 LABEL
# 1) EMPLOYEE BY ZIPCODE
try:
    if lyr_zipcode.supports("SHOWLABELS"):
        if lyr_zipcode.showLabels:
            for lblClass in lyr_zipcode.listLabelClasses():
                lblClass.expression = "\"<CLR red='67' green='82' blue='84'><FNT name='Tw Cen MT Condensed' size = '15'>\" + $feature.Join_Count + \"</FNT></CLR>\""
                lblClass.SQLQuery = "$feature.Join_Count > 1"
                lblClass.visible = True
            lyr_zipcode.showLabels = True 
except:
    print("ERROR: LABEL | Employee by Zipcode")
else:
    print("Labeling for employee by zipcode is completed successfully!")
# 3. LAYOUT AND EXPORT
# 3.1 MAPFRAMES AND EXTENT
try:
    # Set layout and mapframes
    lyt = aprx.listLayouts()[0]
    mf1 = lyt.listElements("MAPFRAME_ELEMENT", "Map Frame")[0] # MapFrame - Main (New York Metropolitan)
    mf2 = lyt.listElements("MAPFRAME_ELEMENT", "Inset1")[0] # MapFrame - Inset1 (Manhattan)
    # Assign map to mapframes
    mf1.map = aprxMap
    mf2.map = aprxMap
    # Set map extent
    mf1.camera.setExtent(mf1.getLayerExtent(lyr_zipcode)) # Zoom to employee by zipcode layer
    mf1.camera.scale = 950000
    mf2.camera.setExtent(mf2.getLayerExtent(lyr_zipcode))
    mf2.camera.scale = 220000
except:
    print("ERROR: LAYOUT | MAPFRAME")
else:
    print("Mapframes in layout are created successfully!")

#   3) LAYOUT ELEMENTS
#     A. TITLE, SUBTITLE & TITLE LINE
#       a. TITLE
title = lyt.listElements('TEXT_ELEMENT', 'Title')[0]
title.text = proj_title 
#       b. SUBTITLE
subtitle = lyt.listElements('TEXT_ELEMENT', 'Subtitle')[0]
subtitle.text = proj_subtt
#       c. TITLE LINE
titleLine = lyt.listElements('GRAPHIC_ELEMENT', "Line")[0]
titleLine.elementPositionX = title.elementPositionX + title.elementWidth + 0.3
titleLine.elementWidth = 3.0

You entered a csv file for office sites!
Office site feature is created successfully!
You entered a csv file for employee locations!
Employee point feature is created successfully!
Your search radius for employee points is 50 miles!
Mean center point within the search radius is created successfully!
Employees by zipcode layer and NY regional counts table are created successfully!
Total 5 layers with office are added to map successfully!
Rendering for employee by zipcode is completed successfully!
Rendering for employee mean center is completed successfully!
Rendering for office sites is completed successfully!
Labeling for employee by zipcode is completed successfully!


#### Test.4 - Setting halo property of label using ArcPy and CIM

In [ ]:
import arcpy, os, sys
from arcpy import env
arcpy.env.overwriteOutput = True
aprx = arcpy.mp.ArcGISProject("current")
myMap = aprx.listMaps("Map")[0]
myLayer = myMap.listLayers("*off*")[0]
# Return the layer's CIM definition
l_cim = myLayer.getDefinition('V2')
lc = l_cim.labelClasses[0] # Returns first CIMLabelClass object
# Modify a few boolean properties
l_cim.showMapTips = True  #Turn on map tips for bubble tips to appear
l_cim.selectable = False  #Set the layer to not be selectable
l_cim.expanded = True     #Expand the Layer in the Contents pane
# Update halo properties of text symbol
lc.textSymbol.symbol.haloSize = 2
lc.textSymbol.symbol.haloSymbol = sym
# Update CIM defintion
myLayer.setDefinition(l_cim)
myLayer.showLabels = True

#### Test.3 - Zoom to layer extent in mapframe using Camera

In [5]:
import arcpy, os, sys
from arcpy import env
arcpy.env.overwriteOutput = True
aprx = arcpy.mp.ArcGISProject("current")
aprxMap = aprx.listMaps("Map")[0]
lyt = aprx.listLayouts()[0]
mf1 = lyt.listElements("MAPFRAME_ELEMENT", "Map Frame")[0] # MapFrame - Main (New York Metropolitan)
mf2 = lyt.listElements("MAPFRAME_ELEMENT", "Inset1")[0] # MapFrame - Inset1 (Manhattan)
mf1.map = aprxMap
mf2.map = aprxMap
# Set the layer to zoom-in
lyr_zipcode = aprxMap.listLayers("*zip*")[0]
# Set map extent
mf1.camera.setExtent(mf1.getLayerExtent(lyr_zipcode)) # Zoom to employee by zipcode layer
mf1.camera.scale = 950000
#mf2.camera.setExtent(mf2.getLayerExtent(lyr_zipcode))
mf2.camera.X = -73.97056903072004 # longitude, X
mf2.camera.Y = 40.713446507015554 # Latitude, Y 
mf2.camera.scale = 220000

#### Test.2 - Copy table with only selected fields from feature class using fieldmap

In [3]:
import arcpy, os, sys
from arcpy import env
arcpy.env.overwriteOutput = True
work_dbs = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb"
emp_count = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\Emp_Count"
aprx = arcpy.mp.ArcGISProject("CURRENT") # Current Project
aprxMap = aprx.listMaps()[0] # Default Map

# list of fields I want to keep (capitalization counts!)
myfields = ['Join_Count', 'ID', 'REGION', 'SUBREGION']
# create an empty field mappings object
fms = arcpy.FieldMappings()
# for each field, create an individual field map, and add it to the field mappings object
for field in myfields :
    fm = arcpy.FieldMap()
    fm.addInputField(emp_count, field)
    fms.addFieldMap(fm)
# copy the feature class using the fields you want
arcpy.conversion.TableToTable(
    in_rows = emp_count, 
    out_path = work_dbs, 
    out_name = "emp_count_table", 
    field_mapping = fms)

<Result 'C:\\ArcGIS\\Projects\\Dev_EMA\\Dev_EMA.gdb\\emp_count_table'>

#### Test.1 - Input layers to map using for loop

In [2]:
import arcpy, os, sys
from arcpy import env
arcpy.env.overwriteOutput = True

off_point = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\off_point_test"
emp_meancenter = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\emp_meancenter_test"
emp_zipcode = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\emp_zipcode_test"
emp_count = r"C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\Emp_Count"
cbreMap = r"N:\Team-GISData\Researched_Data\NewYork\Arcpy_Tools\EmployeeMap\Neutral_Basemap_2022.lyrx"
aprx = arcpy.mp.ArcGISProject("CURRENT") # Current Project
aprxMap = aprx.listMaps()[0] # Default Map

import_layers = [cbreMap, emp_count, emp_zipcode, emp_meancenter, off_point]
for layer in import_layers:
    aprxMap.addDataFromPath(layer)